# Tugas Besar IF4074 - Pembelajaran Mesin Lanjut
# Implementasi Convolutional Neural Network

# Simple CNN
**Simple CNN** is a convolutional neural network implemented in Python and fine-tuned using backpropagation algorithm.

## Setup
Assuming you've installed the latest version of Python (if not, guides for it are widely available),
1. ensure pip is installed by running `python -m ensurepip --upgrade`;
2. install the Python dependencies by running `pip install -r requirements.txt`.

## Contribution (Milestone 1)
| NIM      | Name                   | Contribution(s)                                                       |
|----------|------------------------|-----------------------------------------------------------------------|
| 13520041 | Ilham Pratama          | Dataset handling; Detector, Pooling, Dense, and Flatten layer; Report |
| 13520042 | Jeremy S.O.N. Simbolon | Class model; Convolutional layer; Report                              |


### Library Import

In [13]:
import math
import os

from typing import Any

import cv2
import jsonpickle
import jsonpickle.ext.numpy
import numpy as np
import numpy.typing as npt

from scipy.special import expit

### Dataset Loading

In [14]:
class Utils:
    """
    Module related utility functions.

    This class is used to prepare the image dataset for the CNN model. In
    addition, this class is also used to save and load the CNN model.
    """

    @staticmethod
    def load_dataset(dataset_path: str) -> tuple[npt.NDArray, npt.NDArray, dict]:
        """
        Preprocess the dataset and return useful information for further processing.

        :param dataset_path: A string representation of the path pointing to
                             the dataset.
        :return: A tuple consisted of an ndarray of dataset image path, an
                 ndarray of image labels, and a dictionary that maps class
                 labels to folder name.
        """
        folder_list = sorted(os.listdir(dataset_path))
        image_path = []
        image_label = np.array([], dtype=np.int16)
        image_dictionary = {}
        for i, folder_name in enumerate(folder_list):
            class_folder_path = os.path.join(dataset_path, folder_name)
            list_image_name = sorted(os.listdir(class_folder_path))
            temp_folder_path = [os.path.join(class_folder_path, image_name) for image_name in list_image_name]

            image_path += temp_folder_path
            temp_class_label = np.full(len(list_image_name), i, dtype=np.int16)
            image_label = np.concatenate((image_label, temp_class_label), axis=0)
            image_dictionary[str(i)] = folder_name

        return np.asarray(image_path), image_label, image_dictionary

    @staticmethod
    def convert_image_to_matrix(path: npt.NDArray) -> npt.NDArray:
        """
        Convert the image dataset into a list of ndarray.

        Each ndarray is an RGB representation of each image in the dataset.

        :param path: An ndarray of string representation of the path pointing
                     to each image entry in the dataset.
        :return: A list of ndarray representation of the image in the dataset.
        """
        list_of_image_matrix = []
        size = (256, 256)

        for file_img in path:
            image = cv2.imread(file_img, 1)
            matrix = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            matrix = cv2.resize(matrix, size)
            list_of_image_matrix.append(matrix)

        return np.array(list_of_image_matrix)

    @staticmethod
    def save_model(model_object: "Model", file_name: str = "model.json") -> None:
        jsonpickle.ext.numpy.register_handlers()
        with open(file_name, "w") as file:
            json = jsonpickle.encode(model_object, indent=4)
            file.write(json)

    @staticmethod
    def load_model(file_name: str = "model.json") -> "Model":
        jsonpickle.ext.numpy.register_handlers()
        with open(file_name, "r") as file:
            json = file.read()
            return jsonpickle.decode(json)

### Model Representation

In [15]:
class Model:
    """
    The convolutional neural network model used to classify images.
    """

    def __init__(self) -> None:
        """
        Instantiate the convolutional neural network model.
        """
        self._layers = []
        self._result = []

    class Layer:
        """
        Base representation of the layer used as part of the convolutional
        neural network architecture.
        """

        def __init__(self, name) -> None:
            """
            Instantiate the base layer.

            :param name: Name of the layer.
            """
            self._name = name

        def forward_propagate(self) -> None:
            """Indicate the forward propagation is being performed."""
            print(f"Performing forward propagation on {self._name} layer...")
            print()

        def backward_propagate(self) -> None:
            """Indicate the backward propagation is being performed."""
            print(f"Performing backward propagation on {self._name} layer...")
            print()

    class ConvolutionLayer(Layer):
        """
        The convolutional layer in convolutional neural network.

        This class is inherited from the ``Layer`` class. This layer is used
        to perform the convolution operation on the input weights.
        """

        def __init__(
            self,
            filter_count: int,
            filter_size: tuple[int, int] = (32, 32),
            padding_size: int = 0,
            stride_size: tuple[int, int] = (1, 1),
        ) -> None:
            """
            Instantiate the convolutional layer.

            :param filter_count: An integer specifying the amount of feature
                                 to be extracted in the form of the amount of
                                 filters.
            :param filter_size: A tuple of two integers specifying the height
                                and width of the convolution filter.
            :param padding_size: An integer specifying the dimension of 0's to
                                 be added around the weight.
            :param stride_size: A tuple of two integers specifying the pixel
                                step size along the height and width of the
                                input weight.
            """
            super().__init__("convolution")
            self._filter_count = filter_count
            self._filter_dimension = 0
            self._filter_height, self._filter_width = filter_size
            self._filter_weights = None
            self._padding_size = padding_size
            self._stride_height, self._stride_width = stride_size
            self._output_height = 0
            self._output_width = 0
            self._weight_dimension = 0
            self._weight_height = 0
            self._weight_width = 0
            self._biases = None

        def _pad_weights(
            self,
            weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]],
            padding_size: int,
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            """
            Pad the specified weights with 0's around it.

            :param weights: The ndarray of weights to be padded with 0's.
            :param padding_size: An integer specifying the dimension of 0's to
                                 be added around the weight.
            :return: An ndarray of weights padded with 0's.
            """
            self._weight_dimension = len(weights)

            self._weight_height = (weight_height := len(weights[0])) + 2 * padding_size
            self._weight_width = (weight_width := len(weights[0][0])) + 2 * padding_size

            padded_weights = [
                [
                    [
                        weights[i][j - padding_size][k - padding_size]
                        if padding_size <= j < weight_height + padding_size
                        or padding_size <= k < weight_width + padding_size
                        else 0.0
                        for k in range(self._weight_width)
                    ]
                    for j in range(self._weight_height)
                ]
                for i in range(self._weight_dimension)
            ]

            return np.array(padded_weights)

        def convolute(
            self,
            weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]],
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            """
            Perform the convolution operation on the input weights.

            :param weights: An ndarray of input weights.
            :return: An ndarray of features extracted from the weights.
            """
            self._filter_dimension = len(weights)
            self._output_height = (
                math.ceil((len(weights[0]) - self._filter_height + 2 * self._padding_size) / self._stride_height) + 1
            )
            self._output_width = (
                math.ceil((len(weights[0][0]) - self._filter_width + 2 * self._padding_size) / self._stride_width) + 1
            )

            if self._filter_weights is None:
                self._filter_weights = np.random.rand(
                    self._filter_count,
                    self._filter_dimension,
                    self._filter_height,
                    self._filter_width,
                )
            if self._biases is None:
                self._biases = np.random.rand(self._filter_count, self._output_height, self._output_width)

            feature_maps = np.copy(self._biases)
            weights = self._pad_weights(weights, self._padding_size)
            for i in range(self._filter_count):
                for j in range(0, self._weight_height - self._filter_height + 1, self._stride_height):
                    for k in range(0, self._weight_width - self._filter_width + 1, self._stride_width):
                        for l in range(self._filter_dimension):
                            field = weights[l, j : j + self._filter_height, k : k + self._filter_width]
                            feature = field * self._filter_weights[i][l]
                            feature_maps[i][j][k] += np.sum(feature)
            return feature_maps

        def forward_propagate(
            self, weights: npt.NDArray[npt.NDArray[npt.NDArray[float]]]
        ) -> npt.NDArray[npt.NDArray[npt.NDArray[float]]]:
            """
            Indicate and perform the convolution process on the input weights.

            :param weights: The ndarray of weights to be convoluted.
            :return: An ndarray of convoluted weights.
            """
            super().forward_propagate()
            result = self.convolute(weights)
            print("Convolution result: ")
            print(result)
            print()
            return result

        def backward_propagate(self) -> npt.NDArray:
            """
            Indicate and perform the backward propagation operation on the model.
            """
            pass

    class DetectorLayer(Layer):
        """
        The detector layer in convolutional neural network.

        This class is inherited from the ``Layer`` class. This layer is used to
        introduce non-linearity to the learning process using the reLU
        activation function.
        """

        def __init__(self) -> None:
            """Instantiate the detector layer."""
            super().__init__("detector")

        @staticmethod
        def detect(feature: npt.NDArray) -> npt.NDArray:
            """
            Apply the reLU activation function on the input weights.

            :param feature: An ndarray of input weights.
            :return: An ndarray of weights on which the reLU function has been
                     applied.
            """
            return np.maximum(feature, 0)

        def forward_propagate(self, feature: npt.NDArray) -> npt.NDArray:
            """
            Indicate and perform the detector process on the input weights.

            :param feature: The ndarray of weights on which reLU function is
                            to be applied.
            :return: An ndarray of activated weights.
            """
            super().forward_propagate()
            result = self.detect(feature)
            print("Detector result: ")
            print(result)
            print()
            return result

        def backward_propagate(self) -> npt.NDArray:
            """
            Indicate and perform the backward propagation operation on the model.
            """
            pass

    class PoolingLayer(Layer):
        """
        The pooling layer in convolutional neural network.

        This class is inherited from the ``Layer`` class. This layer is used to
        down-sample the input weights according to the specified pooling
        operation.
        """

        def __init__(self, filter_size: int, stride_size: int, mode: str = "max") -> None:
            """
            Instantiate the pooling layer.

            :param filter_size: An integer specifying the dimension of the
                                pooling window.
            :param stride_size: An integer specifying the pixel step size along
                                the height and width of the input weight.
            :param mode: A string specifying the preferred pooling operation.
                         Must either be ``average`` or ``max``.
            """
            super().__init__("pooling")
            self._filter_size = filter_size
            self._stride_size = stride_size
            self._mode = mode

        def average(self, input_matrix: npt.NDArray, d: int, h: int, w: int) -> float:
            """
            Take the average of the input values over the pooling window.

            :param input_matrix: The ndarray of weights on which the operation
                                 is applied.
            :param d: An integer specifying the depth location of the pooling
                      window.
            :param h: An integer specifying the height location of the pooling
                      window.
            :param w: An integer specifying the width location of the pooling
                      window.
            :return: The average of the input values.
            """
            h_start = h * self._stride_size
            w_start = w * self._stride_size
            h_end = h_start + self._filter_size
            w_end = w_start + self._filter_size
            return np.average(input_matrix[d, h_start:h_end, w_start:w_end])

        def max(self, input_matrix: npt.NDArray, d: int, h: int, w: int) -> float:
            """
            Take the maximum of the input values over the pooling window.

            :param input_matrix: The ndarray of weights on which the operation
                                 is applied.
            :param d: An integer specifying the depth location of the pooling
                      window.
            :param h: An integer specifying the height location of the pooling
                      window.
            :param w: An integer specifying the width location of the pooling
                      window.
            :return: The maximum of the input values.
            """
            h_start = h * self._stride_size
            w_start = w * self._stride_size
            h_end = h_start + self._filter_size
            w_end = w_start + self._filter_size
            return np.max(input_matrix[d, h_start:h_end, w_start:w_end])

        def pool(self, input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Perform the pooling operation on the input weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of down-sampled weights.
            """
            depth, height, width = input_matrix.shape
            filter_height = (height - self._filter_size) // self._stride_size + 1
            filter_width = (width - self._filter_size) // self._stride_size + 1
            pooled = np.zeros([depth, filter_height, filter_width], dtype=np.double)
            for d in range(0, depth):
                for h in range(0, filter_height):
                    for w in range(0, filter_width):
                        if self._mode == "average":
                            pooled[d, h, w] = self.average(input_matrix, d, h, w)
                        elif self._mode == "max":
                            pooled[d, h, w] = self.max(input_matrix, d, h, w)
            return pooled

        def forward_propagate(self, input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Indicate and perform the pooling operation on the input weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of down-sampled weights.
            """
            super().forward_propagate()
            result = self.pool(input_matrix)
            print("Pooling result: ")
            print(result)
            print()
            return result

        def backward_propagate(self) -> npt.NDArray:
            """
            Indicate and perform the backward propagation operation on the model.
            """
            pass

    class DenseLayer(Layer):
        """
        The dense layer in convolutional neural network.

        This class is inherited from the ``Layer`` class. This layer is used to
        abstractly represent the input data using its weights.
        """

        def __init__(self, unit_count: int, activation: str = "sigmoid") -> None:
            """
            Instantiate the dense layer.

            :param unit_count: An integer specifying the dimension of the
                               output space.
            :param activation: The activation function to be applied to each
                               node. Must either be ``sigmoid`` or ``relu``.
            """
            super().__init__("dense")
            self._unit_count = unit_count
            self._activation = activation
            self._bias = np.zeros(unit_count)
            self._weight = np.random.randn(unit_count)
            self._deltaW = np.zeros((unit_count))

        def dense(self, input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Perform the linear combination and activation of the input weights
            using the layer's weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of linearly-combined and activated weights.
            """
            self.input = input_matrix
            result = np.zeros(self._unit_count)

            for i in range(self._unit_count):
                input_weight = np.sum(self._weight[i] * input_matrix)
                result[i] = input_weight + self._bias[i]

            if self._activation == "sigmoid":
                self.ouput =  expit(result)
            elif self._activation == "relu":
                self.ouput = np.maximum(result, 0)
            return self.ouput

        def forward_propagate(self, input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Indicate and perform the linear combination and activation of the
            input weights using the layer's weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of linearly-combined and activated weights.
            """
            super().forward_propagate()
            result = self.dense(input_matrix)
            print("Dense result: ")
            print(result)
            print()
            return result

        def backward_propagate(self, error) -> npt.NDArray:
            """
            Indicate and perform the backward propagation operation on the model.
            """
            derivative_value = np.array([])
            for i in range(self.output):
                derivative_value = np.append(derivative_value, self.derivative_act_func(self._activation, i))
            
            self._deltaW += np.multiply(derivative_value, error)
            dE = np.matmul(error, self._weight)
            return dE
        
        def update_weight(self, learning_rate, momentum) -> None:
            """ 
            Indicate and perform the update weight and bias on the model
            Update weight formula = w  + momentum * w + learning_rate * errors * output
            Update bias formula = bias + momentum * bias + learning_rate * errors
            """
            for i in range(self._unit_count):
                self._weight[i] = self._weight[i] - ((momentum * self._weight[i]) + (learning_rate * self._deltaW[i] * self.input))

            self._bias = self._bias - ((momentum * self._bias) + (learning_rate * self._deltaW))
            self._deltaW = np.zeros((self._unit_count))

        def derivative_act_func(self, activation, input):
            """ 
            Take derivative value from activation function and input
            """
            if(activation == "sigmoid"):
                return self.sigmoid_detivative(input)
            else:
                return self.relu_derivative(input)
            
        def sigmoid_detivative(self, input) -> float:
            """ 
            Take derivative value from input
            """
            sigmoid = 1/(1+np.exp(-input))
            return sigmoid * (1 - sigmoid)
        
        def relu_derivative(self, input) -> float:
            """ 
            Take derivative value from input
            """
            if(input >= 0):
                return 1
            else:
                return 0
            
    class FlattenLayer(Layer):
        """
        The flatten layer in convolutional neural network.

        This class is inherited from the ``Layer`` class. This layer is used to
        flatten the input weights.
        """

        def __init__(self) -> None:
            """Instantiate the flatten layer."""
            super().__init__("flatten")

        @staticmethod
        def flatten(input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Perform the flatten operation on the input weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of flatten weights.
            """
            return input_matrix.flatten()

        def forward_propagate(self, input_matrix: npt.NDArray) -> npt.NDArray:
            """
            Indicate and perform the flatten operation on the input weights.

            :param input_matrix: An ndarray of input weights.
            :return: An ndarray of flatten weights.
            """
            super().forward_propagate()
            result = self.flatten(input_matrix)
            print("Flatten result: ")
            print(result)
            print()
            return result

        def backward_propagate(self) -> npt.NDArray:
            """
            Indicate and perform the backward propagation operation on the model.
            """
            pass

    def add_layer(self, name: str, **kwargs: Any) -> None:
        """
        Sequentially add the specified layer into the model.

        :param name: A string representation of the layer to be added.
        :param kwargs: Layer-related parameters in the form of key-value pairs.
        """
        match name:
            case "convolution":
                self._layers.append(self.ConvolutionLayer(**kwargs))
            case "detector":
                self._layers.append(self.DetectorLayer())
            case "pooling":
                self._layers.append(self.PoolingLayer(**kwargs))
            case "dense":
                self._layers.append(self.DenseLayer(**kwargs))
            case "flatten":
                self._layers.append(self.FlattenLayer())

    def forward_propagate(self, tensor: npt.NDArray) -> None:
        """
        Indicate and perform the forward propagation operation on the model.

        :param tensor: An ndarray of input weights representing the input
                       pictures.
        """
        for layer in self._layers:
            tensor = layer.forward_propagate(tensor)
        print("Forward propagation result: ")
        print(tensor)
        self._result = tensor

    def backward_propagate(self) -> None:
        """
        Indicate and perform the backward propagation operation on the model.
        """
        pass

    def train(self, tensor: npt.NDArray[npt.NDArray], epochs: int = 3, learning_rate: float = 0.01) -> None:
        """
        Fit and train the CNN model.

        :param tensor: An ndarray of representations of the input pictures to
                       be fed into the model.
        :param epochs: An integer specifying the number of training epochs.
        :param learning_rate: A float specifying the learning rate of the model.
        """
        pass

### Test result

In [16]:
folder_path, class_label, class_dictionary = Utils.load_dataset("./dataset")
image_matrix = Utils.convert_image_to_matrix(folder_path)
image_number = 0
image_matrix = [image_matrix[image_number]]

model = Model()
model.add_layer(
    "convolution",
    filter_count=32,
    filter_size=(3, 3),
    padding_size=0,
    stride_size=(1, 1),
)
model.add_layer("detector")
model.add_layer("pooling", filter_size=3, stride_size=2, mode="average")
model.add_layer("flatten")
model.add_layer("dense", unit_count=8, activation="relu")
model.add_layer("dense", unit_count=1, activation="sigmoid")
model.forward_propagate(image_matrix)

Performing forward propagation on convolution layer...

Convolution result: 
[[[ 518.00623867  508.50634669  504.70159231 ...  487.77704907
    484.17426784  483.48092541]
  [ 511.24694066  504.82296992  503.1682234  ...  488.21547317
    484.99003603  484.58338992]
  [ 508.53080276  502.96791226  503.55985291 ...  490.54979017
    486.52522003  486.49967256]
  ...
  [ 270.14084207  198.83162194  110.52850671 ...   18.89156003
     47.39773508  115.07108251]
  [ 175.29199377  107.96770598   84.02170649 ...   48.95276385
     54.49732937  139.06712246]
  [ 141.64669208   77.43524397   62.40921933 ...   58.05035492
     70.98258445  126.37645669]]

 [[ 627.34826602  621.06464208  618.96991105 ...  594.62775067
    592.68395951  596.52093486]
  [ 619.82939605  615.85721079  616.61729198 ...  595.80292911
    593.63197716  596.4843313 ]
  [ 617.40990408  615.07190876  618.17959486 ...  596.8930948
    593.78026144  596.19487861]
  ...
  [ 215.08956487  156.49611652  141.14996625 ...   36.6